In [28]:
# !pip install https://github.com/pyg-team/pytorch_geometric.git

In [29]:
# !git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [30]:
%cd hackaton/

[Errno 2] No such file or directory: 'hackaton/'
/home/valerio/Desktop/noisy_labels/hackaton


/home/valerio/.pyenv/versions/3.10.15/envs/hackaton/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [31]:
# !gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


In [32]:
!ls -lh datasets

total 4.0K
drwxrwxr-x 6 valerio valerio 4.0K May 25 13:03 data


In [33]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse


# Set the random seed
set_seed()


In [34]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [35]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [36]:
from sklearn.metrics import f1_score
from tqdm import tqdm
import torch

def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            logits = model(data)
            pred = logits.argmax(dim=1)

            if calculate_accuracy:
                loss = criterion(logits, data.y)
                total_loss += loss.item()
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)

                all_preds.extend(pred.cpu().tolist())
                all_labels.extend(data.y.cpu().tolist())
            else:
                all_preds.extend(pred.cpu().tolist())

    if calculate_accuracy:
        avg_loss = total_loss / len(data_loader)
        accuracy = correct / total
        macro_f1 = f1_score(all_labels, all_preds, average='macro')
        return avg_loss, accuracy, macro_f1

    return all_preds


In [37]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [38]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [39]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = ""#input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [40]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)", default = "./datasets/data/A/train.json.gz")
    args['test_path'] = get_user_input("Path to the test dataset",default = "./datasets/data/A/test.json.gz")
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=0, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin-virtual')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.1, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=50, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE)", default=2, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


In [41]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Arguments received:
train_path: ./datasets/data/A/train.json.gz
test_path: ./datasets/data/A/test.json.gz
num_checkpoints: None
device: 0
gnn: gin-virtual
drop_ratio: 0.1
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 50
baseline_mode: 2
noise_prob: 0.2


In [42]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [43]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()

if args.baseline_mode == 2:
    criterion = NoisyCrossEntropyLoss(args.noise_prob)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [44]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [45]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [46]:
def plot_all_metrics(train_losses, train_accuracies, val_losses, val_accuracies, save_dir):
    """
    Plots and saves a single figure containing training/validation loss and accuracy over epochs.
    """
    os.makedirs(save_dir, exist_ok=True)
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(10, 6))
    # Loss curves
    plt.plot(epochs, train_losses, label='Train Loss', linestyle='-')
    plt.plot(epochs, val_losses, label='Val Loss', linestyle='--')
    # Accuracy curves
    plt.plot(epochs, train_accuracies, label='Train Acc', linestyle='-.')
    plt.plot(epochs, val_accuracies, label='Val Acc', linestyle=':')

    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Training and Validation Metrics')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    save_path = os.path.join(save_dir, 'all_metrics.png')
    plt.savefig(save_path)
    plt.close()
    print(f"All metrics plot saved to {save_path}")


In [ ]:
from collections import Counter

train_paths = ["./datasets/data/A/train.json.gz", 
              "./datasets/data/B/train.json.gz", 
              "./datasets/data/C/train.json.gz", 
              "./datasets/data/D/train.json.gz"]

test_paths = ["./datasets/data/A/test.json.gz", 
              "./datasets/data/B/test.json.gz", 
              "./datasets/data/C/test.json.gz", 
              "./datasets/data/D/test.json.gz"]

for train_path, test_path in zip(train_paths, test_paths):

    args.train_path = train_path
    args.test_path = test_path
    print(args.train_path, args.test_path)
    test_dir_name = os.path.basename(os.path.dirname(args.test_path))
    print(f"Processing dataset: {test_dir_name}")
    continue
    logs_folder = os.path.join(script_dir, "logs", test_dir_name)
    log_file = os.path.join(logs_folder, "training.log")
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
    logging.getLogger().addHandler(logging.StreamHandler())

    checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
    checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
    os.makedirs(checkpoints_folder, exist_ok=True)

    if args.train_path:
        # Carico l’intero dataset e lo splitto

        full_dataset = GraphDataset(args.train_path, transform=add_zeros)
        val_size = int(0.2 * len(full_dataset))
        train_size = len(full_dataset) - val_size

        generator = torch.Generator().manual_seed(12)
        train_dataset, val_dataset = random_split(
            full_dataset, [train_size, val_size], generator=generator
        )

        labels = [d.y.item() for d in train_dataset]
        label_counts = Counter(labels)
        total = len(labels)
        num_classes = len(label_counts) 

        # weights = torch.tensor([total / (num_classes * label_counts[c]) for c in range(num_classes)], 
        #                        dtype=torch.float,
        #                        device=device)

        # if args.baseline_mode == 1:
        #     criterion = torch.nn.CrossEntropyLoss(weight=weights)

        train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=4)

        num_epochs = 2
        best_val_accuracy = 0.0   
        # best_f1 = 0.0
        # best_epoch = -1
        # patience = 0
        # wait = 0

        train_losses = []
        train_accuracies = []
        val_losses = []
        val_accuracies = []

        if num_checkpoints > 1:
            checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
        else:
            checkpoint_intervals = [num_epochs]

        for epoch in range(num_epochs):
            train_loss, train_acc = train(
                train_loader, model, optimizer, criterion, device,
                save_checkpoints=(epoch + 1 in checkpoint_intervals),
                checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
                current_epoch=epoch
            )

            val_loss, val_acc, val_f1 = evaluate(val_loader, model, device, calculate_accuracy=True)
            print(
                f"Epoch {epoch+1}/{num_epochs} | "
                f"Train Acc: {train_acc:.4f} | "
                f"Val Acc: {val_acc:.4f} | "
                f"Val Macro-F1: {val_f1:.4f} | "
                f"Val Loss: {val_loss:.4f}"
            )
            logging.info(
                f"Epoch {epoch + 1}/{num_epochs}, "
                f"Loss: {train_loss:.4f}, "
                f"Train Acc: {train_acc:.4f}, "
                f"Val Acc: {val_acc:.4f}, "
                f"Val Macro-F1: {val_f1:.4f}, "
                f"Val Loss: {val_loss:.4f}"
            )


            #print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

            train_losses.append(train_loss)
            train_accuracies.append(train_acc)
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)

            # if val_f1 > best_f1 + 1e-4:
            #     best_epoch = epoch + 1
            #     best_f1 = val_f1
                
            if val_acc > best_val_accuracy:
                best_val_accuracy = val_acc
                torch.save(model.state_dict(), checkpoint_path)
                print(f"Best model updated and saved at {checkpoint_path}")

        plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
        plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))
        plot_all_metrics(train_losses, train_accuracies, val_losses, val_accuracies, os.path.join(logs_folder, "plots_all_1"))


    import gc
    del train_dataset
    del train_loader
    del full_dataset
    del val_dataset
    del val_loader
    gc.collect()

    test_dataset = GraphDataset(args.test_path, transform=add_zeros)
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
    save_predictions(predictions, args.test_path)

./datasets/data/A/train.json.gz ./datasets/data/A/test.json.gz
./datasets/data/B/train.json.gz ./datasets/data/B/test.json.gz
./datasets/data/C/train.json.gz ./datasets/data/C/test.json.gz
./datasets/data/D/train.json.gz ./datasets/data/D/test.json.gz


In [48]:
# test_dataset = GraphDataset(args.test_path, transform=add_zeros)
# test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [49]:
# model.load_state_dict(torch.load(checkpoint_path, map_location=device))
# predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
# save_predictions(predictions, args.test_path)

In [50]:
# import os
# from collections import Counter
# import torch
# from torch.utils.data import random_split
# import numpy as np

# # Imposta root_dir alla cartella che contiene A/, B/, C/, D/
# root_dir = '/home/valerio/Desktop/noisy_labels/hackaton/datasets/data'
# # oppure, se passi --train_path uguale a questo path:
# # root_dir = args.train_path

# log_file = "stats.log"
# # Pulisci il log all’avvio
# with open(log_file, "w") as f:
#     f.write("")

# for subname in sorted(os.listdir(root_dir)):
#     subdir = os.path.join(root_dir, subname)
#     train_path = os.path.join(subdir, "train.json.gz")

#     if not os.path.isdir(subdir) or not os.path.isfile(train_path):
#         print(f"Skipping {subdir}, not a valid dataset directory or missing train.json.gz")
#         continue

#     print(f"Processing dataset {subname}")

#     # Carico e splitto
#     full_dataset = GraphDataset(train_path, transform=add_zeros)
#     val_size = int(0.2 * len(full_dataset))
#     train_size = len(full_dataset) - val_size
#     generator = torch.Generator().manual_seed(12)
#     train_dataset, val_dataset = random_split(
#         full_dataset, [train_size, val_size], generator=generator
#     )

#     # Calcolo distribuzione
#     labels = [d.y.item() for d in train_dataset]
#     counts = Counter(labels)
#     total = len(labels)

#     with open(log_file, "a") as f:
#         f.write(f"=== Dataset {os.path.basename(subdir)} ===\n")
#         for cls in sorted(counts):
#             cnt = counts[cls]
#             pct = cnt / total * 100
#             f.write(f"Label {cls}: {cnt} esempi ({pct:.1f}%)\n")

#         f.write("\n--- Esempi struttura grafo ---\n")
#         for cls in sorted(counts):
#             sample = next(d for d in train_dataset if d.y.item() == cls)
#             f.write(
#                 f"Classe {cls}: num_nodes={sample.num_nodes}, "
#                 f"num_edges={sample.num_edges}\n"
#             )

#         # Statistiche aggregate
#         f.write("\n--- Statistiche aggregate per classe ---\n")
#         f.write("cls\tn_samp\tnodes_mean\tnodes_std\tedges_mean\tedges_std\n")
#         for cls in sorted(counts):
#             samples = [d for d in train_dataset if d.y.item() == cls]
#             nodes = [d.num_nodes for d in samples]
#             edges = [d.num_edges for d in samples]
#             f.write(
#                 f"{cls}\t{counts[cls]}\t"
#                 f"{np.mean(nodes):.1f}\t{np.std(nodes):.1f}\t"
#                 f"{np.mean(edges):.1f}\t{np.std(edges):.1f}\n"
#             )

#         f.write("\n\n")

#     # Una volta processata questa cartella, non serve cercare altri train.json.gz in subdirectory
#     # se sai che i dataset sono solo A–D direttamente sotto root_dir. Altrimenti commenta questa linea.
#     # break  


In [51]:
# import torch.nn.functional as F
# class SCELoss(torch.nn.Module):
#     def __init__(self, num_classes, alpha=0.1, beta=1.0):
#         super().__init__()
#         self.alpha, self.beta = alpha, beta
#         self.num_classes = num_classes

#     def forward(self, logits, targets):
#         #CCE
#         ce  = F.cross_entropy(logits, targets, reduction='none')

#         #RCE
#         pred = F.softmax(logits, dim=1).clamp(min=1e-6, max=1-1e-6)
#         one_hot = F.one_hot(targets, self.num_classes).float()
#         rce = -(1 - one_hot) * torch.log(1 - pred)
#         rce = rce.sum(dim=1)
#         return (self.alpha * ce + self.beta * rce).mean()